# 04 Predictive Modeling (Simple and Explainable)

## Objectives

- Build a simple, explainable forecast model
- Evaluate performance with time-aware splits
- Export predictions for the dashboard

## Inputs

- data/processed/v1/environmental_trends_clean.csv

## Outputs

- data/processed/v1/model_predictions.csv

## Additional Comments

- Report limitations and avoid overclaiming

---

# Change working directory

In [ ]:
import os
current_dir = os.getcwd()
os.chdir(os.path.dirname(current_dir))
os.getcwd()

# Load processed data

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
clean_path = "data/processed/v1/environmental_trends_clean.csv"
df = pd.read_csv(clean_path)
df.head()

# Prepare features and target (placeholder)

In [ ]:
# Replace column names to match dataset
# Example: predict Avg_Temperature_degC using Year and other signals
# df_model = df.dropna(subset=["Year", "Avg_Temperature_degC"])
# X = df_model[["Year"]]
# y = df_model["Avg_Temperature_degC"]


# Time-aware split and model training (placeholder)

In [ ]:
# Example time split
# split_year = 2018
# train = df_model[df_model["Year"] <= split_year]
# test = df_model[df_model["Year"] > split_year]
# model = LinearRegression()
# model.fit(train[["Year"]], train["Avg_Temperature_degC"])
# preds = model.predict(test[["Year"]])
# mae = mean_absolute_error(test["Avg_Temperature_degC"], preds)
# rmse = mean_squared_error(test["Avg_Temperature_degC"], preds, squared=False)
# mae, rmse

# Export predictions (placeholder)

In [ ]:
# Example export
# preds_df = test[["Year"]].copy()
# preds_df["Predicted_Avg_Temperature_degC"] = preds
# preds_path = "data/processed/v1/model_predictions.csv"
# preds_df.to_csv(preds_path, index=False)
# preds_path